## 실습04번: 현재 내 위치에서 가장 가까운 식당 찾고 시각화

#### 새로운 조건이 등장했습니다. "현재 내 위치" , "가장 가까운"

이번 실습은 저희가 모은 망고플레이트 맛집 데이터에서 현재 위치기반 가장 가까운 식당을 찾고자 하는 상황을 가정합니다.

지금까지 웹 크롤링, 위/경도 변환, 지도시각화의 과정을 거쳤으니

이제 추가적인 요구사항을 반영하는 일만 남았습니다.

빠른 실행을 위해 쓸데없는 주석은 삭제하도록하겠습니다.

In [ ]:
import pandas as pd

데이터 수집은 실습03까지 해놓은 데이터로도 가능할 것같습니다.

In [ ]:
restaurant_df = pd.read_csv('./한대앞_망고플레이트_맛집목록_위경도.csv')
# 칼럼명을 소문자 latitude, longitude로 변경
restaurant_df.columns = ['name', 'address', 'latitude', 'longitude']
restaurant_df.head(5)

#### 현재 위치의 주소를 입력하면 위/경도를 뱉어내는 Google API이용

실습03번 위/경도 변환 2번째 방법인 google geocode API를 기억하시나요?

한글 주소를 입력하면 위/경도값을 list로 반환하는 함수를 작성했었습니다. 

이번에도 사용해보죠.

In [ ]:
import requests
import json

def get_latitude_longitude(addr):
    '''
    DESCRIPTION: 주소를 입력하면 위도(latitude)/경도(longitude)를 반환하는 함수 
    ex) get_latitude_longitude('경기도 안산시 상록구 한양대학로 55')
    '''
    location = addr
    response = requests.get('http://maps.googleapis.com/maps/api/geocode/json?sensor=false&language=ko&address=' + location)    
    loc_dict = json.loads(response.text)
    latitude = loc_dict['results'][0]['geometry']['location']['lat'] 
    longitude = loc_dict['results'][0]['geometry']['location']['lng']
    return [latitude, longitude]

#### 현재 내 위치 : 한양대 에리카캠퍼스 경상관

주소: 경기도 안산시 상록구 사3동 한양대학로55 한양대학교 경상대학

In [ ]:
# 주소를 기반으로 위/경도 데이터 얻기
my_location = get_latitude_longitude('경기도 안산시 상록구 사3동 한양대학로55 한양대학교 경상대학')

### 현 위치를 알아냈으니, 이제 '가장 가까운' 식당을 찾아봅시다.

위/경도를 아는 상태에서의 두 점 간의 거리를 측정하는 방법에는 

삼각함수의 haversine이 있습니다.

괜찮습니다. 저도 모릅니다. 이런 상황에서 이런 도구를 쓴다는것이 중요하죠!

In [ ]:
import math
def haversine(my_location, target_location):
    '''
    DESCRIPTION: 두 위도경도 사이의 대원거리(great circle distance)를 계산. 위도경도는 각도 단위로 계산.
    '''
    latitude1, longitude1 = my_location
    latitude2, longitude2 = target_location
    lat1, lon1, lat2, lon2 = map(math.radians, [latitude1, longitude1, latitude2, longitude2])
    
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1
    
    a = math.sin(d_lat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    meter = 6367 * c * 1000
    return round(meter, 2)
    

#### 현재 위치기반 모든 식당의 거리를 재봅시다.

In [ ]:
distance_list = []
for r in restaurant_df[['latitude', 'longitude']].values:
    distance_list.append(haversine(my_location, r))

In [ ]:
distance_list[:3]

이제 모든 각 식당들이 현재 제 위치에서 몇 미터나 떨어졌는지 알아냈습니다. 

이걸 새로 데이터 테이블에 정리해봅시다.

In [ ]:
restaurant_df['distance'] = distance_list

In [ ]:
restaurant_df.head()

이제 가장 작은 distance값을 찾으면 되겠군요?

pandas의 정렬법은 sort_values라는 method가 있습니다.

In [ ]:
near_restaurant_df = restaurant_df.sort_values('distance')
near_restaurant_df.reset_index(drop=True, inplace=True)
near_restaurant_df.head(5)


현 위치기반 가장 가까운 식당이 어딘가요? 이제 지도 시각화를 해봅시다.

In [ ]:
nearest_latlng = near_restaurant_df.loc[0][['latitude', 'longitude']].values

In [ ]:
import folium
# 지도 
hanyang_univ_map = folium.Map(location=[  37.300853, 126.842930], zoom_start=15)
# 식당표시
folium.CircleMarker(nearest_latlng, color='red', fill_color='red', radius=5).add_to(hanyang_univ_map)
# 내 위치 표시
folium.CircleMarker(my_location, color='blue', fill_color='blue', radius=5).add_to(hanyang_univ_map)
hanyang_univ_map